# ReporterScreen sample / guide quality report

Examine the quality of the guide and samples and masks the low-quality guides and samples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import perturb_tools as pt
import bean as be
from bean.qc.utils import fill_in_missing_samples

plt.style.use("default")

In [ ]:
exp_id = "LDLRCDS"
save_format = "png"
bdata_path = "../results/mapped/LDLRCDS/bean_count_LDLRCDS_combined.h5ad"
out_bdata_path = "../results/mapped/LDLRCDS/bean_count_LDLRCDS_masked.h5ad"
rel_pos_is_reporter=False
edit_quantification_start_pos = 2
edit_quantification_end_pos = 7
target_pos_col = "target_pos"
count_correlation_thres = 0.7
edit_rate_thres = 0.1
posctrl_col = "group"
posctrl_val = "PosCtrl"
lfc_thres = -0.1
replicate_label = "replicate"
condition_label = "condition"
comp_cond1 = "top"
comp_cond2 = "bot"
ctrl_cond = "bulk"
recalculate_edits = True
tiling = None
base_edit_data = True
remove_bad_replicates = False

In [ ]:
bdata = be.read_h5ad(bdata_path)

In [ ]:
if tiling is not None:
    bdata.uns['tiling'] = tiling
elif 'tiling' in bdata.uns:
    tiling = bdata.uns['tiling']
else:
    raise ValueError("Ambiguous assignment if the screen is a tiling screen. Provide `--tiling=True` or `tiling=False`.")

In [ ]:
if not isinstance(replicate_label, str):
    bdata.uns["sample_covariates"] = replicate_label[1:]
bdata.samples["replicate"] = bdata.samples[replicate_label] = bdata.samples[replicate_label].astype(str)
bdata.samples["condition"] = bdata.samples[condition_label]

Add dummy samples if not paired

In [ ]:
bdata = fill_in_missing_samples(bdata, condition_label, replicate_label)

In [ ]:
bdata.samples

In [ ]:
bdata.guides

Annotate unannotated samples & log-normalize guides

In [ ]:
#bdata.samples[[replicate_label, condition_label]] = bdata.samples.index.to_series().str.split("_", expand=True)

In [ ]:
bdata.log_norm()

## Sample quality

### Visualize quality metrics

#### 1. Guide coverage

In [ ]:
pt.qc.plot_guide_coverage(bdata, figsize=(6,4))


In [ ]:
plt.style.use('default')
pt.qc.plot_X_gini(bdata)
plt.savefig(f"{exp_id}_gini.{save_format}")

#### 2. Guide abundance correlation

In [ ]:
pt.qc.plot_correlation(bdata, "Spearman")

#### 3. LFC correlation of positive controls

In [ ]:
selected_guides = bdata.guides[posctrl_col] == posctrl_val if posctrl_col else ~bdata.guides.index.isnull()
print(f"Calculating LFC correlation of {sum(selected_guides)} {'positive control' if posctrl_col else 'all'} guides.")

In [ ]:
ax = pt.qc.plot_lfc_correlation(
    bdata,
    selected_guides,
    method="Spearman",
    cond1=comp_cond1,
    cond2=comp_cond2,
    rep_col=replicate_label,
    compare_col=condition_label,
    figsize=(10, 10),
)

ax.set_title("top/bot LFC correlation, Spearman")
plt.yticks(rotation=0)
plt.xticks(rotation=90)
plt.show()

#### 4. Guide editing rates

In [ ]:
if "target_base_change" not in bdata.uns or not base_edit_data:
    bdata.uns["target_base_change"] = ""
    base_edit_data = False
    print("Not a base editing data or target base change not provided. Passing editing-related QC")
    edit_rate_threshold = -0.1
elif recalculate_edits or "edits" not in bdata.layers.keys() or bdata.layers['edits'].max() == 0:
    if 'allele_counts' in bdata.uns.keys():
        bdata.uns['allele_counts'] = bdata.uns['allele_counts'].loc[bdata.uns['allele_counts'].allele.map(str) != ""]
        bdata.get_edit_from_allele()
        bdata.get_edit_mat_from_uns(
            rel_pos_start=edit_quantification_start_pos, 
            target_pos_col=target_pos_col,
            rel_pos_end=edit_quantification_end_pos, 
            rel_pos_is_reporter=rel_pos_is_reporter
        )

In [ ]:
if "target_base_change" not in bdata.uns or not base_edit_data:
    print(
        "Not a base editing data or target base change not provided. Passing editing-related QC"
    )
elif "edits" in bdata.layers.keys():
    bdata.get_guide_edit_rate(
        editable_base_start=edit_quantification_start_pos,
        editable_base_end=edit_quantification_end_pos,
        condition_col=condition_label,
        unsorted_condition_label=ctrl_cond,
    )
    be.qc.plot_guide_edit_rates(bdata)

### 5. Variant coverage

In [ ]:
if not tiling:
    n_guides = bdata.guides.groupby("target").size()
    int_bins = np.arange(min(0.5, n_guides.min() - 0.5), n_guides.max() + 0.5, 1)
    plt.hist(n_guides, bins=int_bins)
    plt.xticks(np.arange(n_guides.min() - 1, n_guides.max() + 1, 1))
    plt.title("# Guides per target")

In [ ]:
if "target_base_change" not in bdata.uns or not base_edit_data:
    print(
        "Not a base editing data or target base change not provided. Passing editing-related QC"
    )
elif not tiling:
    total_edits = bdata.guides.groupby("target")["edit_rate"].sum()
    plt.hist(n_guides)
    plt.title("Total edit rates per target")

### Mask low-quality samples

In [ ]:
bdata.samples.style.background_gradient(cmap="coolwarm_r")

Assign sample mask to mask low-quality samples.

In [ ]:
bdata.samples["mask"] = 1
bdata.samples.loc[
    ("PosCtrl" in bdata.guides.target_group) & bdata.samples.median_corr_X.isnull() | (bdata.samples.median_corr_X < count_correlation_thres),
    "mask",
] = 0
if "mean_editing_rate" in bdata.samples.columns.tolist():
    bdata.samples.loc[bdata.samples.mean_editing_rate < edit_rate_thres, "mask"] = 0

bdata.samples.loc[
    bdata.samples[f"median_lfc_corr.{comp_cond1}_{comp_cond2}"] < lfc_thres,
    "mask",
] = 0
if posctrl_val in bdata.guides[posctrl_col].values:
    print("filter with posctrl LFC")
    bdata.samples.loc[
        bdata.samples[f"median_lfc_corr.{comp_cond1}_{comp_cond2}"].isnull(),
        "mask",
    ] = 0

In [ ]:
bdata.samples.style.background_gradient(cmap="coolwarm_r")

In [ ]:
# leave replicate with more than 1 sorting bin data
if remove_bad_replicates:
    rep_n_samples = bdata.samples.groupby(replicate_label)["mask"].sum()
    print(rep_n_samples)
    rep_has_too_small_sample = rep_n_samples.loc[rep_n_samples < 2].index.tolist()
    print(
        f"Excluding reps {rep_has_too_small_sample} that has less than 2 samples per replicate."
    )
    if isinstance(replicate_label, str):
        samples_include = ~bdata.samples[replicate_label].isin(
            rep_has_too_small_sample
        )
    else:
        bdata.samples["_rc"] = bdata.samples[
            replicate_label
        ].values.tolist()
        samples_include = ~bdata.samples["_rc"].isin(rep_has_too_small_sample)
        bdata.samples.pop("_rc")
    bdata_filtered = bdata[:, samples_include]
    if isinstance(replicate_label, str) and len(bdata_filtered.samples[replicate_label].unique()) <= 1 or isinstance(replicate_label, list) and len(bdata_filtered.samples[replicate_label].drop_duplicates() <= 1): 
        raise ValueError("Too small number of replicate left after QC. Check the input data or adjust the QC metric thresholds.")
else:
    bdata_filtered = bdata

In [ ]:
bdata_filtered.samples.style.background_gradient(cmap="coolwarm_r")

## Identify outlier guides

In [ ]:
outlier_guides, mask = be.qc.get_outlier_guides_and_mask(bdata_filtered, condit_col = condition_label, replicate_col = replicate_label)

In [ ]:
outlier_guides


In [ ]:
outlier_guides_n_samples = outlier_guides['name'].value_counts()
guides_to_exclude = outlier_guides_n_samples.loc[outlier_guides_n_samples > 2].index
guides_to_exclude

In [ ]:
bdata_filtered.uns['repguide_mask'] = mask

In [ ]:
bdata_filtered = bdata_filtered[~bdata_filtered.guides.index.isin(guides_to_exclude),:]

In [ ]:
bdata_filtered

In [ ]:
bdata_filtered.uns['repguide_mask'].shape

In [ ]:
bdata_filtered.write(out_bdata_path)